# Project 02 - Image Processing

## Thông tin sinh viên

- Họ và tên: Nguyễn Thọ Tài
- MSSV: 23127255
- Lớp: 23CLC02

## Import các thư viện liên quan

In [12]:
!python -m pip install matplotlib
!python -m pip install numpy
!python -m pip install Pillow

from PIL import Image # for read, write image
import numpy as np # for matrix compute
import matplotlib.pyplot as plt # for show image
import colorsys # for convert RGB to HSL

## Helper functions

In [23]:
# Any optional parameters beyond the required ones should be defined with default values
def read_img(img_path):
	""" Read image from img_path
	returns a 2D image (numpy array)
	"""
	return np.array(Image.open(img_path))

def show_img(img_2d):
	""" Show image
	"""
	plt.imshow(img_2d)
	plt.axis('off')
	plt.show()

def save_img(img_2d, img_path: str):
	"""	Save image to img_path
	"""
	Image.fromarray(img_2d).save(img_path)

def convert_rgb_to_hsl_loop(img_2d):
    """ Convert RGB image to HSL image
    returns a 2D image (numpy array)
    """
    norm_img = np.array(img_2d) / 255.0
    height, width, _ = norm_img.shape
    hsl_array = np.zeros_like(norm_img) # since (h,s,l) shares the same space as (r, g, b)

    for i in range(height):
        for j in range(width):
            r, g, b = norm_img[i, j]
            h, l, s = colorsys.rgb_to_hls(r, g, b)
            hsl_array[i, j] = [h, s, l]

    return hsl_array

def convert_hsl_to_rgb_loop(img_2d):
    """ Convert HSL image to RGB image
    returns a 2D image (numpy array)
    """
    height, width, _ = img_2d.shape

    rgb_array = np.zeros_like(img_2d)

    for i in range(height):
        for j in range(width):
            h, s, l = img_2d[i, j]
            r, g, b = colorsys.hls_to_rgb(h, l, s)
            rgb_array[i, j] = [r, g, b]

    rgb_array = (rgb_array * 255).astype(np.uint8)
    return Image.fromarray(rgb_array)

def convert_rgb_to_hsl(img_2d):
	""" Convert RGB image to HSL image
	returns a 2D image (numpy array)
	"""
	norm_img = np.array(img_2d) / 255.0
	rgb2hsl_vec = np.vectorize(colorsys.rgb_to_hls)
	h, l, s = rgb2hsl_vec(norm_img[..., 0], norm_img[..., 1], norm_img[..., 2]) # img[..., 0] means take all previous dim but only the 0 index of the last one. (4, 4, 3) -> (4, 3)
	return np.stack([h, s, l], axis=-1)

def convert_hsl_to_rgb(img_2d):
	""" Convert HSL image to RGB image
	returns a 2D image (numpy array)
	"""
	hsl2rgb_vec = np.vectorize(colorsys.hls_to_rgb)
	r, g, b = hsl2rgb_vec(img_2d[..., 0], img_2d[..., 1], img_2d[..., 2])
	return Image.fromarray((np.stack([r, g, b], axis=-1) * 255).astype(np.uint8))

def img_change_brightness(img_2d, brightness_factor):
	pass

def img_change_contrast(img_2d, contrast_factor):
	pass

def img_flip_horizontal(img_2d):
	return img_2d[:, ::-1]

def img_flip_vertical(img_2d):
	return img_2d[::-1, :]

def img_flip_horizontal_vertical(img_2d):
	return img_2d[::-1, ::-1]

def img_crop(img_2d, start: list[int], end: list[int]):
	height, width, _ = img_2d.shape
	x1, x2 = sorted([max(0, min(start[0], width)), max(0, min(end[0], width))])
	y1, y2 = sorted([max(0, min(start[1], height)), max(0, min(end[1], height))])

	return img_2d[y1:y2, x1:x2, :]

def img_crop_quarter_center(img_2d):
	height, width, _ = img_2d.shape
	start_height, start_width = height // 4, width // 4
	return img_crop(img_2d, [start_width, start_height], [start_width * 3, start_height * 3])

def blank_mask(img_2d):
	h, w, c = img_2d.shape
	masked_image = np.zeros_like(img_2d)
	if c == 4: # if RGBA image, set the mask to full opaque (img_2d.max() -> 1.0 or 255)
		masked_image[..., 3] = img_2d.max()
	return masked_image

def img_circle_mask(img_2d):
	height, width, _ = img_2d.shape
	Y, X = np.mgrid[0:height, 0:width]
	cx, cy = width // 2, height // 2
	r = min(cx, cx)

	mask = (X - cx) ** 2 + (Y - cy) ** 2 <= r ** 2
	masked_img = blank_mask(img_2d)
	masked_img[mask] = img_2d[mask]
	return masked_img

def rotated_ellipse_mask(x, y, a, b, theta):
	cos_t, sin_t = np.cos(theta), np.sin(theta)
	term1 = ((x * cos_t + y * sin_t) ** 2) / a ** 2
	term2 = ((x * sin_t - y * cos_t) ** 2) / b ** 2
	return (term1 + term2) <= 1 # x^2 / a^2 + y^2 / b^2 <= 1, take the inner part

def img_2ellipse_mask(img_2d: np.ndarray):
	height, width, _ = img_2d.shape
	Y, X = np.ogrid[:height, :width]
	diag = (width * width + height * height) ** 0.5 - 5

	a, b = 0.875 * diag / 2, 0.5 * diag / 2
	theta = np.deg2rad(45)

	# translate the grid coord to the center
	x = X - width // 2
	y = Y - height // 2

	frame_mask = np.logical_or(rotated_ellipse_mask(x, y, a, b, theta),
							   rotated_ellipse_mask(x, y, a, b, -theta))

	masked_img = blank_mask(img_2d)
	masked_img[frame_mask] = img_2d[frame_mask]
	return masked_img

MAT_BOX_BLUR = 0.1111111111111111 * np.array(
	[[1, 1, 1],
	 [1, 1, 1],
	 [1, 1, 1]])
MAT_GAUSS_3_BLUR = 0.0625 * np.array(
	[[1, 2, 1],
	 [2, 4, 2],
	 [1, 2, 1]])
MAT_GAUSS_5_BLUR = 0.00390625 * np.array(
	[[1,  4,  6,  4,  1],
	 [4, 16, 24, 16, 14],
	 [6, 24, 36, 24,  6],
	 [4, 16, 24, 16,  4],
	 [1,  4,  6,  4,  1]])
MAT_UNSHARP_MASK_BLUR = -0.00390625 * np.array(
	[[1,  4,    6,  4,  1],
	 [4, 16,   24, 16, 14],
	 [6, 24, -476, 24,  6],
	 [4, 16,   24, 16,  4],
	 [1,  4,    6,  4,  1]])

def kernel_convolution(kernel: np.ndarray, img_2d):
	if kernel.ndim != 2:
		raise ValueError('Input kernel must be 2D')
	if img_2d.ndim != 3:
		raise ValueError('Input image must be 3D')

	norm_img = img_2d.astype(np.float32) / 255.0
	height, width, _ = norm_img.shape
	k_height, k_width = kernel.shape
	if k_height != k_width:
		raise ValueError('Input kernel matrix must be square')

	# pad image to handle boundaries
	pad_size = k_height // 2
	pad_img = np.pad(norm_img, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode='constant')

	patches = np.lib.stride_tricks.sliding_window_view(pad_img, window_shape=(k_height, k_height), axis=(0, 1)) # shape: (height, width, 1, 1, channel, k_height, k_width)
	# As sliding window view creating 2 degenerate dimensions of size 1, we need to remove it from the patches by cutting it off using transpose and sliding
	print(f"Patches shape 1: {patches.shape}\n")
	# patches = patches.transpose(0, 1, 4, 2, 3)  # shape: (height, width, channel, 1, 1, k_height, k_width)
	# print(f"Patches shape 2: {patches.shape}\n")
	# patches = patches[:, :, :, 0, 0]  # shape: (height, width, channel, k_height, k_width)
	# print(f"Patches shape 3: {patches.shape}\n")

	result = np.sum(patches * kernel[None, None, None, :, :], axis=(3, 4))
	result = np.clip(result * 255, 0, 255)
	return result.astype(np.uint8)


def process_image(img_2d, func=[1, 2, 3,...]):
	""" Process image with a list of functions
	func: a list of functions to apply to the image
	return processed 2D image
	"""










In [24]:
# Example 3x3 RGB image
image = np.array([
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]],
    [[10, 11, 12], [13, 14, 15], [16, 17, 18]],
    [[19, 20, 21], [22, 23, 24], [25, 26, 27]]
], dtype=np.uint8)

# 3x3 identity kernel
kernel_3x3 = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]], dtype=np.float32)

# 5x5 Gaussian blur kernel
kernel_5x5 = np.array([
    [1,  4,  6,  4, 1],
    [4, 16, 24, 16, 4],
    [6, 24, 36, 24, 6],
    [4, 16, 24, 16, 4],
    [1,  4,  6,  4, 1]
], dtype=np.float32) / 256.0

# Test
result_3x3 = kernel_convolution(kernel_3x3, image)
print("3x3 kernel result (red channel):\n", result_3x3[:, :, 0])
result_5x5 = kernel_convolution(kernel_5x5, image)
print("5x5 kernel result (red channel):\n", result_5x5[:, :, 0])

Patches shape 1: (3, 3, 3, 3, 3)

3x3 kernel result (red channel):
 [[ 1  4  7]
 [10 13 16]
 [19 22 25]]
Patches shape 1: (3, 3, 3, 5, 5)

5x5 kernel result (red channel):
 [[ 3  5  4]
 [ 7  9  8]
 [ 7 10  8]]


In [14]:
# import numpy as np
# from numpy.lib.stride_tricks import as_strided
#
# # Define kernels as 3x3 NumPy arrays
# identity = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]], dtype=np.float32)
# edge0 = np.array([[1, 0, -1], [0, 0, 0], [-1, 0, 1]], dtype=np.float32)
# edge1 = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]], dtype=np.float32)
# edge2 = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]], dtype=np.float32)
# sharpen = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32)
# box_blur = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]], dtype=np.float32) * 0.1111
# gaussian_blur = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]], dtype=np.float32) * 0.0625
# emboss = np.array([[-2, -1, 0], [-1, 1, 1], [0, 1, 2]], dtype=np.float32)
#
# def convolution(kernel, image):
#     """
#     Apply a 3x3 kernel to each channel of the image using vectorized convolution.
#
#     Args:
#         kernel: 3x3 NumPy array representing the convolution kernel
#         image: Input image as a NumPy array (height, width, 3)
#
#     Returns:
#         Processed image as a NumPy array (height, width, 3, uint8)
#     """
#     # Normalize image to [0, 1]
#     image = image.astype(np.float32) / 255.0
#     height, width = image.shape[:2]
#
#     # Pad image with zeros for simplicity (can switch to 'edge' if needed)
#     image_padded = np.pad(image, ((1, 1), (1, 1), (0, 0)), mode='constant', constant_values=0)
#
#     # Create sliding window view
#     shape = (height, width, 3, 3, 3)  # (height, width, channels, kernel_height, kernel_width)
#     strides = image_padded.strides[:2] + (image_padded.strides[2],) + image_padded.strides[:2]
#     patches = as_strided(image_padded, shape=shape, strides=strides)
#
#     # Apply kernel to all patches and channels in one step
#     result = np.sum(patches * kernel[None, None, None, :, :], axis=(3, 4))
#
#     # Clip and convert to uint8
#     result = np.clip(result, 0, 1) * 255
#     return result.astype(np.uint8)
#
# def main():
#     # Load image using imageio (or assume image is provided)
#     try:
#         import imageio
#         image = imageio.imread('input.jpg')
#     except ImportError:
#         print("Please install imageio (`pip install imageio`) or provide a NumPy array image.")
#         return
#     except FileNotFoundError:
#         print("Input image 'input.jpg' not found. Please provide a valid image file.")
#         return
#
#     # Ensure image is RGB
#     if image.shape[2] == 4:  # Convert RGBA to RGB if needed
#         image = image[:, :, :3]
#
#     # Apply convolution with the emboss kernel
#     processed_image = convolution(emboss, image)
#
#     # Save the output image
#     imageio.imwrite('output_emboss.jpg', processed_image)
#
# if __name__ == "__main__":
#     main()

## Your tests

In [15]:
 # YOUR CODE HER
import os

def test_flip():
	os.makedirs("flip/", exist_ok=True)
	img = read_img("demo.png")
	flip_h_img = img_flip_horizontal(img)
	flip_v_img = img_flip_vertical(img)
	flip_hv_img = img_flip_horizontal_vertical(img)
	save_img(flip_h_img, "flip/flip_horizontal_demo.png")
	save_img(flip_v_img, "flip/flip_vertical_demo.png")
	save_img(flip_hv_img, "flip/flip_horizontal_vertical_demo.png")

def test_mask():
	os.makedirs("mask/", exist_ok=True)
	img = read_img("cat.jpg")
	circle_masked_img = img_circle_mask(img)
	ellipse_masked_img = img_2ellipse_mask(img)
	save_img(circle_masked_img, "mask/mask_circle_cat.png")
	save_img(ellipse_masked_img, "mask/mask_ellipse_cat.png")

test_flip()
test_mask()

## Main FUNCTION

In [16]:
# YOUR CODE HERE